<a href="https://colab.research.google.com/github/AcerPing/AceTibaMe/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload() #upload kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle (1).json


In [ ]:
!kaggle competitions download -c titanic

401 - Unauthorized


In [ ]:
'''
from urllib request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0716/raw/master/train.csv"
urlretrieve(url,"train.csv")
url = "https://github.com/Elwing-Chou/ml0716/raw/master/test.csv"
urlretrieve(url,"test.csv")
'''


'\nfrom urllib request import urlretrieve\nurl = "https://github.com/Elwing-Chou/ml0716/raw/master/train.csv"\nurlretrieve(url,"train.csv")\nurl = "https://github.com/Elwing-Chou/ml0716/raw/master/test.csv"\nurlretrieve(url,"test.csv")\n'

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv",encoding="utf-8")
test_df = pd.read_csv("test.csv",encoding="utf-8")
# train_df

In [ ]:
datas = pd.concat([train_df,test_df])
datas = datas.drop(["PassengerId","Survived"],axis=1)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
# Step1. 補缺失值
s = datas.isna().sum()
# 篩選 series[[True...]]
s[s > 0]

Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [ ]:
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [ ]:
# 補數字類型[中位數]
med = datas.median().drop(["Pclass"])
datas = datas.fillna(med)
datas.isna().sum()

Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin       1014
Embarked       0
dtype: int64

In [ ]:
#先準備一下我要保留的list
def name(n):
  n = n.split(",")[-1].split(".")[0]
  return n.strip()
v = datas["Name"].apply(name).value_counts()
white = v.index[:4]
white

Index(['Mr', 'Miss', 'Mrs', 'Master'], dtype='object')

In [ ]:
def name(n):
  n = n.split(",")[-1].split(".")[0]
  n = n.strip()
  if n in white:
    return n
  else:
    return None
datas["Name"] = datas["Name"].apply(name)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,28.0,0,0,A.5. 3236,8.0500,NaN,S
414,1,None,female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,Mr,male,28.0,0,0,359309,8.0500,NaN,S


In [ ]:
v = datas["Ticket"].value_counts()
def ticket(t):
  global v
  return v[t]
datas["Ticket"] = datas["Ticket"].apply(ticket)

In [ ]:
datas = datas.drop(["Cabin"],axis=1)
# 如果你想要重置index
datas.reset_index(drop=True)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,Mr,male,22.0,1,0,1,7.2500,S
1,1,Mrs,female,38.0,1,0,2,71.2833,C
2,3,Miss,female,26.0,0,0,1,7.9250,S
3,1,Mrs,female,35.0,1,0,2,53.1000,S
4,3,Mr,male,35.0,0,0,1,8.0500,S
...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,28.0,0,0,1,8.0500,S
414,1,None,female,39.0,0,0,3,108.9000,C
415,3,Mr,male,38.5,0,0,1,7.2500,S
416,3,Mr,male,28.0,0,0,1,8.0500,S


In [ ]:
datas = pd.get_dummies(datas)

In [ ]:
#取列: iloc(看實際是第幾列),loc(看列標籤)
#[第一列,第二列...]
x_train = datas.iloc[:len(train_df)]
x_test = datas.iloc[len(train_df):]
y_train = train_df["Survived"]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier(random_state=0)
params = {"n_estimators":range(20,100),
      "max_depth":range(5,10)}
search = GridSearchCV(clf,params,cv=10,n_jobs=-1)
search.fit(x_train,y_train)
print(search.best_params_)
print(search.best_score_)

{'max_depth': 6, 'n_estimators': 78}
0.8395255930087391


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
clf = RandomForestClassifier(n_estimators=77,max_depth=7,random_state=0)
scores = cross_val_score(clf,x_train,y_train,cv=10,n_jobs=-1)
print(scores)
print(np.average(scores))

[0.78888889 0.86516854 0.74157303 0.8988764  0.86516854 0.82022472
 0.83146067 0.7752809  0.87640449 0.83146067]
0.829450686641698


In [ ]:
clf = RandomForestClassifier(n_estimators=77,max_depth=7)
clf.fit(x_train,y_train)
pre = clf.predict(x_test)
df = pd.DataFrame({"PassengerID":test_df["PassengerId"],
          "Survived":pre})
df.to_csv("titanic_rf.csv",encoding="utf-8",index=False)
df

,PassengerID,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
pd.DataFrame({
    "Name":datas.columns,
    "Importance":clf.feature_importances_
})

,Name,Importance
0,Pclass,0.102238
1,Age,0.089660
2,SibSp,0.041890
3,Parch,0.031599
4,Ticket,0.075846
5,Fare,0.132278
6,Name_Master,0.015864
7,Name_Miss,0.024653
8,Name_Mr,0.157618
9,Name_Mrs,0.033237


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
datas_scale = pd.DataFrame(scaler.fit_transform(datas),columns=datas.columns)
# data_scale
x_train_scale = datas_scale.iloc[:len(train_df)]
x_predict_scale = datas_scale.iloc[len(train_df):]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
params = {"n_neighbors":range(5,100)}
search = GridSearchCV(clf,params,cv=10,n_jobs=-1)
search.fit(x_train_scale,y_train)
print(search.best_params_)
print(search.best_score_)

{'n_neighbors': 16}
0.8204369538077403


In [ ]:
clf = search.best_estimator_
pre = clf.predict(x_predict_scale)
df = pd.DataFrame({"PassengerID":test_df["PassengerId"],
          "Survived":pre})
df.to_csv("titanic_knn.csv",encoding="utf-8",index=False)
df

,PassengerID,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
